In [ ]:
import itertools
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
import pandas as pd
import config
import os
import random

In [ ]:
def all_orderings_all_splits(df, cluster_column='cluster_label', label_name='KL-Score'):
    """
    For each KL-based binary split, return PR-AUC for ALL permutations
    of cluster label orderings.
    """

    # Unique cluster IDs
    clusters = sorted(df[cluster_column].unique())
    K = len(clusters)

    # All permutations of cluster label orderings
    permutations = list(itertools.permutations(range(K)))

    # Define severity splits
    thresholds = {
        "auc_pr":  lambda x: x > 0,   # KL > 0
        "auc_mid": lambda x: x > 1,   # KL > 1
        "auc_mid2": lambda x: x > 2,  # KL > 2
        "auc_sev": lambda x: x == 4   # KL == 4
    }

    # Final output
    results = {}

    for key, condition in thresholds.items():
        y_true = (condition(df[label_name])).astype(int).values

        # Ensure binary
        if not set(np.unique(y_true)).issubset({0,1}):
            raise ValueError(f"y_true for {key} split is not binary.")    

        split_results = []

        # Evaluate AUC for EVERY permutation
        for perm in permutations:
            # Build mapping: cluster_id → severity_score_from_perm
            mapping = {cluster: score for cluster, score in zip(clusters, perm)}

            # Convert cluster label to continuous score
            y_score = df[cluster_column].map(mapping).values.astype(float)

            # Compute PR-AUC
            precision, recall, _ = precision_recall_curve(y_true, y_score)
            pr_auc = auc(recall, precision)

            split_results.append({
                "mapping": mapping,
                "auc": pr_auc
            })

        results[key] = split_results

    return results

In [ ]:
STAGE = 'ss'
MOD_PREFIX = "mod_smallimg3"
NEPOCH = 'latest'


DATAPATH = config.OUTPUT_PATH
base_dir = config.RAW_DATA_PATH
img_path = config.SCHULTHESS_DATAPATH
proc_dir = config.PROC_DATA_PATH

# #for rawq:
# feature = 'rawq'
# folder = "2025-11-19_hdbscan"
# run = "run10"  

# feature = 'img_features'
# #for img features:
# folder = "2025-09-12_hdbscan"
# folder_date = folder.split('_')[0]
# run = "run92"

# feature = 'img_raw'
# folder = "2025-09-13_hdbscan_img"
# run = "run32"

feature = 'agg'
folder = "2025-08-11_hdbscan"
run = 'run150'

anomalyscore_metric = "centre_mean"
cluster_col = "cluster_label"

folder_date = folder.split('_')[0]



if feature == 'rawq':
    filepath = os.path.join(proc_dir, folder, "pipeline", run)
    hdbscan_df = pd.read_csv(os.path.join(filepath, f'pipeline_{run}_umap_hdbscan_scaled.csv'))
elif feature == 'img_features' or feature == 'img_raw':
        filepath = os.path.join(proc_dir, "radiographic_features", folder, run)
        hdbscan_df = pd.read_csv(os.path.join(filepath, f'{folder}_{run}_umap_hdbscan_scaled.csv'))
elif feature == 'agg':
      filepath = os.path.join(proc_dir, folder, run)
      hdbscan_df = pd.read_csv(os.path.join(filepath, f'{folder}_{run}_umap_hdbscan_scaled.csv'))
        
kl = pd.read_csv(os.path.join(base_dir,  "brul_knee_annotations.csv"))
kl2 = pd.read_csv(os.path.join(base_dir, "rosand1_knee_annotations.csv"))
mri = pd.read_csv(os.path.join(base_dir, '2025-09-25_mrismall.csv'))

# with open(os.path.join(filepath, f'pipeline_{run}_umap_hdbscan_scaled_model_info.json')) as f:
#     model_info= json.load(f)

In [ ]:
hdbscan_df = hdbscan_df.merge(kl, left_on = 'id', right_on='name', how='left', validate='one_to_one')
hdbscan_df = hdbscan_df.merge(kl2, left_on = 'id', right_on='name', how='left', validate='one_to_one', suffixes=('', '2'))

try:
    hdbscan_df.drop(columns=['Unnamed: 0'], inplace=True)
except:
    pass

df = pd.read_csv(os.path.join(DATAPATH, 'outputs', 'dfs', 'ss', 'mod_smallimg3_ss_aggregated_scores.csv'))

df['id_temp'] = df['id'].apply(lambda x: x.split('/')[-1])
df['id'] = df['id_temp'].apply(lambda x: x.split('.')[0])
df.drop(columns=['id_temp'], inplace=True)
hdbscan_df = hdbscan_df.merge(df, on='id', how='left', validate='one_to_one')

# if 'KL-Score'  is na, fill with 'KL-Score2'
hdbscan_df['KL-Score'] = hdbscan_df['KL-Score'].fillna(hdbscan_df['KL-Score2'])
hdbscan_df['KL-Score2'] = hdbscan_df['KL-Score2'].fillna(hdbscan_df['KL-Score'])

hdbscan_df['KL-Score'].fillna(-1, inplace=True)
hdbscan_df['KL-Score2'].fillna(-1, inplace=True)

hdbscan_df_dropna = hdbscan_df.copy()
hdbscan_df_dropna = hdbscan_df_dropna[hdbscan_df_dropna['KL-Score'] != -1]
hdbscan_df_dropna = hdbscan_df_dropna[hdbscan_df_dropna['KL-Score2'] != -1]

In [ ]:
hdbscan_df_dropna['cluster_label'].value_counts()


In [ ]:
# results = all_orderings_all_splits(hdbscan_df_dropna, 'cluster_label', 'KL-Score')

In [ ]:
df= hdbscan_df_dropna.copy()

label_name='KL-Score'
cluster_column='cluster_label'


clusters = sorted(df[cluster_column].unique())
K = len(clusters)

# All permutations of cluster label orderings
# permutations = list(itertools.permutations(range(K)))
permutations = []

for _ in range(50):
    permutations.append(random.sample(clusters, len(clusters)))

# Define severity splits
thresholds = {
    "auc_pr":  lambda x: x > 0,   # KL > 0
    "auc_mid": lambda x: x > 1,   # KL > 1
    "auc_mid2": lambda x: x > 2,  # KL > 2
    "auc_sev": lambda x: x == 4   # KL == 4
}

In [ ]:
results = {}
i = 0
for perm in permutations:
    i += 1
    mapping = {cluster: score for cluster, score in zip(clusters, perm)} #cluster: klscore

    y_score = df[cluster_column].map(mapping).values.astype(float)

    results[i] = {'perm' :mapping, 'recall-precision': {}}

    for key, condition in thresholds.items():
        y_true = (condition(df[label_name])).astype(int).values

        # Ensure binary
        if not set(np.unique(y_true)).issubset({0,1}):
            raise ValueError(f"y_true for {key} split is not binary.")
        
        precision, recall, _ = precision_recall_curve(y_true, y_score)
        pr_auc = auc(recall, precision)
        pr_auc = float('{:.1f}'.format(pr_auc*100))

        #print(f'{key} PR-AUC: {(pr_auc*100):.1f}')
        results[i]['recall-precision'][key] = pr_auc
        # results[i][key] = pr_auc

In [ ]:
def get_top_k(results, k=5):
    # extract metric names from the first entry
    sample_key = next(iter(results))
    metrics = results[sample_key]["recall-precision"].keys()

    top_k_indices = {}

    for metric in metrics:
        # Build a list of (index, value)
        values = [(idx, res["recall-precision"][metric]) 
                  for idx, res in results.items()]

        # Sort by the value (descending = best values first)
        values_sorted = sorted(values, key=lambda x: x[1], reverse=True)

        # Select top k indices
        top_k_indices[metric] = values_sorted[:k]

    return top_k_indices
top_k_results = get_top_k(results, k=5)

In [ ]:
def print_top_k(results, k=5):
    # get metric names from the first entry
    sample_key = next(iter(results))
    metrics = results[sample_key]["recall-precision"].keys()

    for metric in metrics:
        print(f"\n=== Top {k} permutations for {metric} ===")

        # Build list of (index, value, perm)
        values = []
        for idx, res in results.items():
            auc_value = res["recall-precision"][metric]
            perm = res["perm"]
            values.append((idx, auc_value, perm))

        # Sort by AUC descending
        values_sorted = sorted(values, key=lambda x: x[1], reverse=True)

        # Print top k
        for rank, (idx, auc_value, perm) in enumerate(values_sorted[:k], start=1):
            print(f"\n#{rank}  (perm index: {idx})")
            print(f"AUC: {auc_value}")
            print(f"Permutation mapping: {perm}")


In [ ]:
top_k_results

In [ ]:
print_top_k(results, k=5)

In [ ]:
def best_orderings_auc_pr_and_sev(df, permutations, cluster_column='cluster_label', label_name='KL-Score'):
    """
    Computes PR-AUC across all permutations of cluster orderings.
    Returns ONLY the best permutation for:
        • auc_pr  (KL > 0)
        • auc_sev (KL == 4)
    Also returns all scores in case needed.
    """

    import itertools
    import numpy as np
    from sklearn.metrics import precision_recall_curve, auc

    # Unique cluster IDs
    clusters = sorted(df[cluster_column].unique())
    K = len(clusters)

    # # All permutations
    # permutations = list(itertools.permutations(range(K)))

    # Define severity splits
    thresholds = {
        "auc_pr":  lambda x: x > 0,   # KL > 0
        "auc_mid": lambda x: x > 1,   # KL > 1
        "auc_mid2": lambda x: x > 2,  # KL > 2
        "auc_sev": lambda x: x == 4   # KL == 4
    }

    # Final dictionary
    all_results = {}
    best_results = {
        "auc_pr":  {"best_auc": -1, "best_mapping": None},
        "auc_sev": {"best_auc": -1, "best_mapping": None}
    }

    for key, condition in thresholds.items():
        y_true = (condition(df[label_name])).astype(int).values

        if not set(np.unique(y_true)).issubset({0,1}):
            raise ValueError(f"y_true for {key} split is not binary.")

        split_scores = []
        
        for perm in permutations:
            # cluster → score mapping
            mapping = {cluster: score for cluster, score in zip(clusters, perm)}

            # score for PR curve
            y_score = df[cluster_column].map(mapping).astype(float).values

            precision, recall, _ = precision_recall_curve(y_true, y_score)
            pr_auc = auc(recall, precision)

            split_scores.append({
                "mapping": mapping,
                "auc": pr_auc
            })

            # Track best only for auc_pr and auc_sev
            if key in best_results:
                if pr_auc > best_results[key]["best_auc"]:
                    best_results[key]["best_auc"] = pr_auc
                    best_results[key]["best_mapping"] = mapping

        all_results[key] = split_scores

    return {
        "all_results": all_results,
        "best_auc_pr": best_results["auc_pr"],
        "best_auc_sev": best_results["auc_sev"]
    }


In [ ]:
results = best_orderings_auc_pr_and_sev(df,permutations,  'cluster_label', 'KL-Score')

In [ ]:
print(results['best_auc_pr'])
print(results['best_auc_sev'])

In [ ]:
def best_orderings_aucroc_pr_and_sev(df, permutations, cluster_column='cluster_label', label_name='KL-Score'):
    """
    Computes ROC-AUC across all permutations of cluster orderings.
    Returns ONLY the best permutation for:
        • auc_roc (KL > 0)
        • auc_sev (KL == 4)
    Also returns full results.
    """

    import itertools
    import numpy as np
    from sklearn.metrics import roc_curve, auc

    # Unique cluster IDs
    clusters = sorted(df[cluster_column].unique())
    K = len(clusters)

    # # All permutations of orderings
    # permutations = list(itertools.permutations(range(K)))

    # Severity threshold definitions (same as before)
    thresholds = {
        "auc_roc": lambda x: x > 0,   # KL > 0
        "auc_mid": lambda x: x > 1,   # KL > 1
        "auc_mid2": lambda x: x > 2,  # KL > 2
        "auc_sev": lambda x: x == 4   # KL == 4
    }

    # Storage
    all_results = {}
    best_results = {
        "auc_roc": {"best_auc": -1, "best_mapping": None},
        "auc_sev": {"best_auc": -1, "best_mapping": None}
    }

    for key, condition in thresholds.items():
        y_true = (condition(df[label_name])).astype(int).values

        # check binary validity
        if not set(np.unique(y_true)).issubset({0,1}):
            raise ValueError(f"y_true for {key} split is not binary.")

        split_scores = []

        # evaluate every cluster ordering
        for perm in permutations:

            # build mapping: cluster -> score (continuous)
            mapping = {cluster: score for cluster, score in zip(clusters, perm)}
            y_score = df[cluster_column].map(mapping).astype(float).values

            # ROC curve
            fpr, tpr, _ = roc_curve(y_true, y_score)
            roc_auc = auc(fpr, tpr)

            split_scores.append({
                "mapping": mapping,
                "auc": roc_auc
            })

            # Update ONLY for the two metrics we care about
            if key in best_results:
                if roc_auc > best_results[key]["best_auc"]:
                    best_results[key]["best_auc"] = roc_auc
                    best_results[key]["best_mapping"] = mapping

        all_results[key] = split_scores

    return {
        "all_results": all_results,
        "best_auc_roc": best_results["auc_roc"],
        "best_auc_sev": best_results["auc_sev"]
    }


In [ ]:
results = best_orderings_aucroc_pr_and_sev(df, permutations, 'cluster_label', 'KL-Score')
print(results['best_auc_roc'])
print(results['best_auc_sev'])

In [ ]:
def sort_mapping(mapping):
    return dict(sorted(mapping.items(), key=lambda x: x[0]))


In [ ]:
sorted_auc_roc_mapping = sort_mapping(results["best_auc_roc"]["best_mapping"])
sorted_auc_sev_mapping = sort_mapping(results["best_auc_sev"]["best_mapping"])


In [ ]:
def mapping_in_custom_order(result_dict, key, cluster_order):
    """
    Extract mapping from result_dict['best_auc_roc'] or ['best_auc_sev']
    and return the mapping values in the order given by cluster_order.

    Parameters
    ----------
    result_dict : output of best_orderings_aucroc_pr_and_sev
    key : 'best_auc_roc' or 'best_auc_sev'
    cluster_order : list of cluster labels in the order you want

    Returns
    -------
    List of scores in the given cluster order.
    """

    mapping = result_dict[key]["best_mapping"]

    # invert: score -> cluster
    inverse = {score: cluster for cluster, score in mapping.items()}

    # return labels sorted by score
    return [inverse[i] for i in sorted(inverse.keys())]


In [ ]:
sorted_auc_sev_mapping

In [ ]:
cluster_order = sorted(df['cluster_label'].unique())

In [ ]:
cluster_order

In [ ]:
mapping_in_custom_order(results, "best_auc_sev", cluster_order)